# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [5]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [6]:
#Cleaning the missing data 
# Justification of techniques used to account for missing data

In [7]:
# Reload original dataset fresh for Q2
df = pd.read_csv("data/beach_sensors.csv")

rows_before = df.shape[0]
print("Rows before cleaning:", rows_before)


Rows before cleaning: 196627


In [8]:
raw_df = pd.read_csv("data/beach_sensors.csv")

raw_df.dtypes


Station Name                    object
Measurement Timestamp           object
Air Temperature                float64
Wet Bulb Temperature           float64
Humidity                         int64
Rain Intensity                 float64
Interval Rain                  float64
Total Rain                     float64
Precipitation Type             float64
Wind Direction                   int64
Wind Speed                     float64
Maximum Wind Speed             float64
Barometric Pressure            float64
Solar Radiation                  int64
Heading                        float64
Battery Life                   float64
Measurement Timestamp Label     object
Measurement ID                  object
dtype: object

In [9]:
raw_df["Measurement Timestamp"].head()
raw_df["Measurement Timestamp"].dtype


dtype('O')

In [10]:
df.describe(percentiles=[0.01, 0.99])
df[["Air Temperature", "Wind Speed", "Solar Radiation"]].quantile([0.001, 0.999])


,Air Temperature,Wind Speed,Solar Radiation
0.001,-19.67,0.0,-6.0
0.999,34.10,12.4,964.0


In [11]:
# Check relationship between Total Rain missing & Interval Rain
missing_rain_mask = df["Total Rain"].isna()

df.groupby(missing_rain_mask)["Interval Rain"].describe()


,count,mean,std,min,25%,50%,75%,max
Total Rain,,,,,,,,
False,120522.0,0.151701,1.077572,-0.9,0.0,0.0,0.0,50.40
True,76105.0,0.127243,1.124669,0.0,0.0,0.0,0.0,63.42


In [12]:
# Convert timestamp to datetime
df["Measurement Timestamp"] = pd.to_datetime(df["Measurement Timestamp"], errors="coerce")

# Force numeric columns to numeric (coerce errors to NaN)
numeric_cols = [
    "Air Temperature", "Wet Bulb Temperature", "Humidity",
    "Rain Intensity", "Interval Rain", "Total Rain",
    "Wind Direction", "Wind Speed", "Maximum Wind Speed",
    "Barometric Pressure", "Solar Radiation", "Heading",
    "Battery Life"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

print("✅ Data types validated and converted")


✅ Data types validated and converted


In [13]:
# Justification of techniques used to account for missing data
#Low missing <0.1% - Rows were dropped
#High missing eg 38% Forward filling was used

# Columns with high missingness (~38%)
high_missing = [
    "Wet Bulb Temperature", "Rain Intensity",
    "Total Rain", "Precipitation Type", "Heading"
]

# Forward-fill high-missing columns (time-series appropriate)
df = df.sort_values("Measurement Timestamp")
df[high_missing] = df[high_missing].ffill()

# Low-missing numeric columns → drop remaining NaNs
df = df.dropna(subset=["Air Temperature", "Barometric Pressure"])

print("✅ Missing values handled")


✅ Missing values handled


In [18]:
#Outlier Detection 
outlier_cols = [
    "Air Temperature", "Humidity",
    "Wind Speed", "Maximum Wind Speed",
    "Barometric Pressure", "Solar Radiation"
]

outlier_report = {}

for col in outlier_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 3 * IQR
    upper = Q3 + 3 * IQR

    outliers = ((df[col] < lower) | (df[col] > upper)).sum()

    df[col] = df[col].clip(lower, upper)

    outlier_report[col] = {
        "lower_bound": lower,
        "upper_bound": upper,
        "outliers_capped": int(outliers)
    }

print("✅ Outliers capped using IQR")


✅ Outliers capped using IQR


In [19]:
#Removing duplicates
duplicates_before = df.duplicated().sum()
df = df.drop_duplicates()
duplicates_after = df.duplicated().sum()

print("Duplicates removed:", duplicates_before)


Duplicates removed: 0


In [20]:
#Cleaned CSV file saved
df.to_csv("output/q2_cleaned_data.csv", index=False)
print("✅ output/q2_cleaned_data.csv saved")


✅ output/q2_cleaned_data.csv saved


In [23]:
#Total Number of rows after cleaning 
rows_after = df.shape[0]

with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(rows_after))

print("✅ output/q2_rows_cleaned.txt saved:", rows_after)

✅ output/q2_rows_cleaned.txt saved: 196479


In [24]:
#Cleaned report
report_lines = []

report_lines.append("DATA CLEANING REPORT")
report_lines.append("====================\n")

report_lines.append(f"Rows before cleaning: {rows_before}\n")

report_lines.append("Missing Data Handling:")
report_lines.append("- Wet Bulb Temperature, Rain Intensity, Total Rain, Precipitation Type, Heading")
report_lines.append("  Method: Forward-fill (time series appropriate)")
report_lines.append("  Reason: High non-random sensor-based missingness\n")

report_lines.append("- Air Temperature, Barometric Pressure")
report_lines.append("  Method: Row-wise removal for remaining NaNs")
report_lines.append("  Reason: Missing rate < 0.1%\n")

report_lines.append("Outlier Handling (IQR Method, 3×IQR):")
for col, stats in outlier_report.items():
    report_lines.append(
        f"- {col}: capped at [{stats['lower_bound']:.2f}, {stats['upper_bound']:.2f}], "
        f"outliers capped = {stats['outliers_capped']}"
    )

report_lines.append(f"\nDuplicates Removed: {duplicates_before}")

report_lines.append("\nData Type Conversions:")
report_lines.append("- Measurement Timestamp → datetime64[ns]")
report_lines.append("- Meteorological variables → numeric")

report_lines.append(f"\nRows after cleaning: {rows_after}")

with open("output/q2_cleaning_report.txt", "w") as f:
    f.write("\n".join(report_lines))

print("✅ output/q2_cleaning_report.txt saved")


✅ output/q2_cleaning_report.txt saved
